

## Types of data

* *slug*_songs.json: List of all the songs for that band. It contains metadata about the songs such as number of times played
* *slug*_years.json: List of every year shows were played and meta data about that those shows.
* *slug*_*year*_shows.json: Every show and source nested similarly to ReListen from that year. Placed in the *years* folder.


* *slug*_shows.csv: Every song, for every source, for every show, for every year. It contains everything, except meta data on ???

Chagnge the slug to the artist to download

In [4]:
import json
import os
import requests
from pathlib import Path
import sys
import time
import urllib

api_url = "https://api.relisten.net/api/v2/"
slug = "grateful-dead"

# Save location
parent_dir = "%s" % slug
try:
    os.mkdir(parent_dir)
except:
    pass

songs_file = Path(parent_dir) / ("%s_songs.json" % slug)
years_file = Path(parent_dir) / ("%s_years.json" % slug)
shows_dir = Path(parent_dir) / "shows"

try:
    os.mkdir(shows_dir)
except:
    pass
    
years = []
shows_dict = {}
combined_table = []


In [5]:
# Download songs

req_url = urllib.parse.urljoin(api_url, "artists/%s/songs"%slug)
r = requests.get (req_url)
print(r)
print(r.url)

parsed = json.loads(r.text)
print(len(parsed))

with open(songs_file, 'w') as f:
    json.dump(parsed, f, indent=4)


<Response [200]>
https://api.relisten.net/api/v2/artists/grateful-dead/songs
557


In [6]:
# Download years

req_url = urllib.parse.urljoin(api_url, "artists/%s/years"%slug)
r = requests.get (req_url)
print(r)
print(r.url)

parsed = json.loads(r.text)
print(len(parsed))

with open(years_file, 'w') as f:
    json.dump(parsed, f, indent=4)

for c in parsed:
    years.append(int(c["year"]))

<Response [200]>
https://api.relisten.net/api/v2/artists/grateful-dead/years
31


In [7]:
# Get show information, broken up by year

start = time.time()
last_time = start
for y in years:
    print("%d: " % y, end="")
    req_url = urllib.parse.urljoin(api_url, "artists/%s/years/%d"%(slug, y))
    r = requests.get (req_url)
    if r.status_code != 200:
        print("Error getting data for year: %d" % y)
    
    year = json.loads(r.text)
    shows = year
    
    for s in range(0, len(year["shows"])):
        req_url = urllib.parse.urljoin(api_url, "artists/%s/years/%d/%s"%(slug, y, year["shows"][s]["display_date"]))
        r = requests.get (req_url)
        if r.status_code != 200:
            print("Error getting data for year: %d; show: %s" % (y, year["shows"][s]["display_date"]))
            print("Status: %d" % r.status_code)

        show = json.loads(r.text)
        shows["shows"][s]["sources"] = show["sources"]

    with open(shows_dir / ("%d_shows.json"%y) , "w") as f:
        #json.dump(shows, f, indent=4)
        json.dump(shows, f)
    
    print("%5.2f, %5.2f" % (time.time()-last_time, time.time()-start))
    last_time = time.time()

1965:  0.50,  0.50
1966:  4.95,  5.45
1967:  3.65,  9.10
1968:  5.52, 14.62
1969: 15.81, 30.43
1970: 13.83, 44.26
1971: 12.53, 56.79
1972: 13.18, 69.97
1973: 12.00, 81.97
1974:  7.45, 89.42
1975:  3.06, 92.48
1976:  6.84, 99.32
1977: 10.01, 109.33
1978: 20.57, 129.90
1979: 20.27, 150.16
1980: 23.72, 173.89
1981: 24.31, 198.20
1982: 17.05, 215.25
1983: 19.72, 234.98
1984: 19.21, 254.19
1985: 21.06, 275.24
1986: 12.84, 288.08
1987: 24.32, 312.41
1988: 21.91, 334.32
1989: 22.52, 356.84
1990: 22.27, 379.11
1991: 21.62, 400.74
1992: 13.76, 414.50
1993: 14.31, 428.81
1994: 13.49, 442.30
1995:  7.45, 449.75


In [ ]:
# Reshape downloaded files into single table

